<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/3_AgregacoesAvancadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=0da2b76d9ac092f9d41846649444e6e2163ea417ab1739976e4ea244aabaa136
  Stored in directory: /root/.cache/pip/wheels/67/c2/7c/a53325365fba358ffff35af84a2e14cf88c18052f88acfa5f0
Successfully built spark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
from google.colab import files
uploaded = files.upload()

Saving avaliacoes_pedido.csv to avaliacoes_pedido.csv
Saving clientes.csv to clientes.csv
Saving itens_pedido.csv to itens_pedido.csv
Saving pagamentos_pedido.csv to pagamentos_pedido.csv
Saving pedidos.csv to pedidos.csv
Saving produtos.csv to produtos.csv
Saving vendedores.csv to vendedores.csv


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spark = SparkSession.builder.getOrCreate()

In [7]:
produtos = spark.read.csv("produtos.csv", header=True, inferSchema=True)
vendedores = spark.read.csv("vendedores.csv", header=True, inferSchema=True)
clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv("avaliacoes_pedido.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("pedidos.csv", header=True, inferSchema=True)

In [8]:
pedidos.show()

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      2018-08-08 13:50:00| 2018-08-17 18:06:29|  2018-09

In [9]:
pedidos.withColumn('compra_anual', year('data_compra_pedido')).withColumn('compra_mensal', month('data_compra_pedido')).groupBy('compra_anual', 'compra_mensal').count().orderBy(desc('compra_anual'), 'compra_mensal').show()

+------------+-------------+-----+
|compra_anual|compra_mensal|count|
+------------+-------------+-----+
|        2018|            1| 7269|
|        2018|            2| 6728|
|        2018|            3| 7211|
|        2018|            4| 6939|
|        2018|            5| 6873|
|        2018|            6| 6167|
|        2018|            7| 6292|
|        2018|            8| 6512|
|        2018|            9|   16|
|        2018|           10|    4|
|        2017|            1|  800|
|        2017|            2| 1780|
|        2017|            3| 2682|
|        2017|            4| 2404|
|        2017|            5| 3700|
|        2017|            6| 3245|
|        2017|            7| 4026|
|        2017|            8| 4331|
|        2017|            9| 4285|
|        2017|           10| 4631|
+------------+-------------+-----+
only showing top 20 rows



In [10]:
avaliacoes_pedido.show(2)
itens_pedido.show(2)
avaliacoes_pedido.join(itens_pedido, 'id_pedido').groupBy('id_produto').agg(format_number(avg('nota_avaliacao'), 2).alias('media_avaliacao')).show()

+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
|        id_avaliacao|           id_pedido|nota_avaliacao|titulo_comentario|mensagem_comentario|data_criacao_avaliacao|data_resposta_avaliacao|
+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|             4|             NULL|               NULL|   2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|             5|             NULL|               NULL|   2018-03-10 00:00:00|    2018-03-11 03:05:13|
+--------------------+--------------------+--------------+-----------------+-------------------+----------------------+-----------------------+
only showing top 2 rows

+--------------------+--------------+--------------------+--------------------+-------------------+-----+------

In [12]:
# Converter timestamp para long(segundos) - Unix timestamp
pedidos_long = pedidos.withColumn('data_compra_pedido_long', unix_timestamp(col('data_compra_pedido')))
pedidos_long = pedidos_long.withColumn('data_aprovacao_pedido_long', unix_timestamp(col('data_aprovacao_pedido')))

# Converter em horas
pedidos_horas = pedidos_long.withColumn('diferenca_horas',
                                        (col('data_aprovacao_pedido_long') - col('data_compra_pedido_long')) / 3600)

pedidos_horas.select('data_compra_pedido', 'data_aprovacao_pedido', 'diferenca_horas').show()

+-------------------+---------------------+-------------------+
| data_compra_pedido|data_aprovacao_pedido|    diferenca_horas|
+-------------------+---------------------+-------------------+
|2017-10-02 10:56:33|  2017-10-02 11:07:15|0.17833333333333334|
|2018-07-24 20:41:37|  2018-07-26 03:24:27|  30.71388888888889|
|2018-08-08 08:38:49|  2018-08-08 08:55:23| 0.2761111111111111|
|2017-11-18 19:28:06|  2017-11-18 19:45:59| 0.2980555555555556|
|2018-02-13 21:18:39|  2018-02-13 22:20:29| 1.0305555555555554|
|2017-07-09 21:57:05|  2017-07-09 22:10:13|0.21888888888888888|
|2017-04-11 12:22:08|  2017-04-13 13:25:17|            49.0525|
|2017-05-16 13:10:30|  2017-05-16 13:22:11| 0.1947222222222222|
|2017-01-23 18:29:09|  2017-01-25 02:50:47|  32.36055555555556|
|2017-07-29 11:55:02|  2017-07-29 12:05:32|              0.175|
|2017-05-16 19:41:10|  2017-05-16 19:50:18|0.15222222222222223|
|2017-07-13 19:58:11|  2017-07-13 20:10:08|0.19916666666666666|
|2018-06-07 10:06:19|  2018-06-09 03:13:

In [15]:
pedidos.withColumn(
    'tempo_aprovacao',
    col('data_aprovacao_pedido').cast('long') - col('data_compra_pedido').cast('long')
).groupBy().agg(
    (avg('tempo_aprovacao')/3600).alias('tempo_medio_aprovacao_horas')
).show()

+---------------------------+
|tempo_medio_aprovacao_horas|
+---------------------------+
|         10.419094301919692|
+---------------------------+



In [18]:
pedidos.select('data_entrega_cliente', 'data_aprovacao_pedido').show(2)

pedidos_antecipados = pedidos.withColumn(
    'entrega_antecipada',
    when(col('data_entrega_cliente') < col('data_estimada_entrega'), 1).otherwise(0)
).groupBy().agg(
    count(when(col('entrega_antecipada') ==1, True)).alias('total_entrega_antecipada')
)

pedidos_antecipados.show()

+--------------------+---------------------+
|data_entrega_cliente|data_aprovacao_pedido|
+--------------------+---------------------+
| 2017-10-10 21:25:13|  2017-10-02 11:07:15|
| 2018-08-07 15:27:45|  2018-07-26 03:24:27|
+--------------------+---------------------+
only showing top 2 rows

+------------------------+
|total_entrega_antecipada|
+------------------------+
|                   88649|
+------------------------+



In [19]:
pedidos.select('data_entrega_cliente', 'data_aprovacaO_pedido').show(2)

tempo_media_envio_status = pedidos.withColumn(
    'tempo_envio',
    col('data_entrega_cliente').cast('long') - col('data_aprovacao_pedido').cast('long')
).groupBy('status_pedido').agg(
    format_number(avg('tempo_envio')/86400, 2).alias('tempo_medio_envio_dias')
)

tempo_media_envio_status.show()

+--------------------+---------------------+
|data_entrega_cliente|data_aprovacaO_pedido|
+--------------------+---------------------+
| 2017-10-10 21:25:13|  2017-10-02 11:07:15|
| 2018-08-07 15:27:45|  2018-07-26 03:24:27|
+--------------------+---------------------+
only showing top 2 rows

+-------------+----------------------+
|status_pedido|tempo_medio_envio_dias|
+-------------+----------------------+
|      shipped|                  NULL|
|     canceled|                 19.91|
|     approved|                  NULL|
|     invoiced|                  NULL|
|      created|                  NULL|
|    delivered|                 12.13|
|  unavailable|                  NULL|
|   processing|                  NULL|
+-------------+----------------------+



In [20]:
pagamentos_pedido.show(2)
itens_pedido.show(2)
pedidos.show(2)
clientes.show(2)

+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|
|a9810da82917af2d9...|                  1|   credit_card|                 1|          24.39|
+--------------------+-------------------+--------------+------------------+---------------+
only showing top 2 rows

+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|           id_pedido|item_id_pedido|          id_produto|         id_vendedor|  data_limite_envio|preco|valor_frete|
+--------------------+--------------+--------------------+--------------------+-------------------+-----+-----------+
|00010242fe8c5a6d1...|             1|4244733e06e7ecb49...|48436

In [21]:
vendas_pedido_join = pagamentos_pedido.join(itens_pedido, 'id_pedido').join(pedidos, 'id_pedido').join(clientes, 'id_cliente')

vendas_por_estado = vendas_pedido_join.groupBy('estado_cliente').agg(sum('valor_pagamento').alias('total_vendas'))
vendas_por_estado.show()

vendas_por_estado.show(2)

+--------------+------------------+
|estado_cliente|      total_vendas|
+--------------+------------------+
|            SC| 786343.7099999998|
|            RO|           65886.0|
|            PI|136779.95999999996|
|            AM|34753.299999999996|
|            RR|          12462.21|
|            GO|513879.00000000035|
|            TO|          72281.17|
|            MT|256804.61999999994|
|            SP| 7597209.659999931|
|            ES| 405805.3400000009|
|            PB| 180984.1899999999|
|            RS|1147277.0000000023|
|            MS|164337.27999999997|
|            AL|         111284.42|
|            MG|2326151.6399999885|
|            PA|261788.34999999992|
|            BA| 797410.3600000021|
|            SE| 88437.50999999998|
|            PE|376377.26999999967|
|            CE| 343847.8300000001|
+--------------+------------------+
only showing top 20 rows

+--------------+-----------------+
|estado_cliente|     total_vendas|
+--------------+-----------------+
|    